In [45]:
import re
path = "/Users/wadood/Library/Mobile Documents/com~apple~CloudDocs/Projects/DataSystems_COMS4121/csds-material/input/"
doc = sc.textFile("file:///"+path)
counts = doc.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)
print(counts)
df_counts = spark.createDataFrame(counts)
df_counts.show()

PythonRDD[388] at RDD at PythonRDD.scala:48
+-------+---+
|     _1| _2|
+-------+---+
|    Bye|  1|
|Goodbye|  1|
|  Hello|  2|
|  World|  2|
| Hadoop|  2|
+-------+---+



In [39]:
from datetime import datetime
from pyspark.sql.types import *
import re

path = "/Users/wadood/Library/Mobile Documents/com~apple~CloudDocs/Projects/DataSystems_COMS4121/csds-material/hive/"
fileName = path+"purchases.txt"
schema = StructType([ \
                   StructField("sales_date",DateType(),True),\
                   StructField("store_location",StringType(),True),\
                   StructField("category",StringType(),True),\
                   StructField("price",FloatType(),True),\
                   StructField("card",StringType(),True)
                  ])
rdd = sc.textFile("file:///"+fileName)\
        .map(lambda x:x.split(',')) \
        .map(lambda x: (datetime.strptime(x[0],'%Y-%m-%d %H:%M:%S'),str(x[1]),str(x[2]),float(x[3]),str(x[4])))
 
        
df = sqlContext.createDataFrame(rdd,schema)
df.registerTempTable("purchases")
#What is the average price of the products that were purchased via Mastercard? */
qry="select avg(Price) from purchases where card='MasterCard'"
result=spark.sql(qry)
result.show()

#2. Which date recorded the highest total sales?
qry ="SELECT to_date(sales_date), SUM(price) AS total_sales FROM purchases GROUP BY to_date(sales_date) SORT BY total_sales DESC LIMIT 1"
result=spark.sql(qry)
result.show()

#3. What is the minimum value of a product under the Computers category? */
qry ="SELECT MIN(price) FROM purchases WHERE category='Computers'"
result=spark.sql(qry)
result.show()

#4. How many distinct categories of products are there? */
qry ="SELECT COUNT(DISTINCT category) FROM purchases"
result=spark.sql(qry)
result.show()

#5. Which store location had the lowest total sales? */
qry ="SELECT store_location, SUM(price) AS store_sales FROM purchases GROUP BY store_location ORDER BY store_sales LIMIT 1"
result=spark.sql(qry)
result.show()


+-----------------+
|       avg(Price)|
+-----------------+
|275.0677317417774|
+-----------------+

+-------------------------------+-----------------+
|to_date(purchases.`sales_date`)|      total_sales|
+-------------------------------+-----------------+
|                     2012-04-17|842.4700012207031|
+-------------------------------+-----------------+

+----------+
|min(price)|
+----------+
|      0.38|
+----------+

+------------------------+
|count(DISTINCT category)|
+------------------------+
|                      18|
+------------------------+

+--------------+-----------------+
|store_location|      store_sales|
+--------------+-----------------+
|         Plano|784.9599838256836|
+--------------+-----------------+

